# Library

In [1]:
# api.py
import duckdb
from fastapi import FastAPI, Query
from pydantic import BaseModel, RootModel
from typing import List, Any


# Fast API Wrapper

## Resampled Data Request

In [ ]:

app = FastAPI()
con = duckdb.connect()  # in-process, zero-config

# Optionally register your Postgres raw table via postgres_scan:
con.execute("""
    INSTALL httpfs; LOAD httpfs;
  /* or: INSTALL postgres_scanner; LOAD postgres_scanner; */
  /* then you can do: */
  /* CREATE VIEW raw AS SELECT * FROM postgres_scan('host=…','public','raw'); */
""")

class Row(RootModel):
    root: List[Any]

class QueryResult(BaseModel):
    columns: List[str]
    rows: List[Row]

@app.get("/resample", response_model=QueryResult)
def resample(
    interval: int = Query(3, description="Resample interval in days"),
    limit: int = Query(50, description="Max rows back")
):
    sql = f"""
    WITH ranked AS (
      SELECT *,
        ROW_NUMBER() OVER (PARTITION BY symbol ORDER BY date) AS rn
        FROM raw
    ),
    grp AS (
      SELECT *,
        (rn - 1) / {interval} AS grp_id
        FROM ranked
    )
    SELECT
        symbol,
        MIN(date)   AS date,
        FIRST(open) AS open,
        MAX(high)   AS high,
        MIN(low)    AS low,
        LAST(close) AS close,
        SUM(volume) AS volume
    FROM grp
    GROUP BY symbol, grp_id
    ORDER BY symbol, date DESC
    LIMIT {limit};
    """
    # execute and grab both column names + native Python lists of tuples
    cur = con.execute(sql)
    cols = [c[0] for c in cur.description]
    data = cur.fetchall()
    return QueryResult(columns=cols, rows=[Row(root=list(r)) for r in data])